In [1]:
import pickle
import os
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
os.chdir('..')
from star_sample import Isochrones,StarSample
os.chdir('Data')
pkl_file = open('isochrones_high_lat.pkl', 'rb')

Isochrone_hl = pickle.load(pkl_file)
os.chdir('..')

In [2]:
os.chdir('Data')
pkl_file = open('high_lat_values.pkl', 'rb')

High_Latitude_sample = pickle.load(pkl_file)
os.chdir('..')

In [3]:
High_Latitude_sample.data['G']=High_Latitude_sample.data['phot_g_mean_mag']-5*np.log10(High_Latitude_sample.data['r_med_geo'])+5

In [4]:
High_Latitude_sample.data['K']=High_Latitude_sample.data['ks_m']-5*np.log10(High_Latitude_sample.data['r_med_geo'])+5

In [5]:
High_Latitude_sample.data['jk']=High_Latitude_sample.data['j_m']-High_Latitude_sample.data['ks_m']

In [6]:
High_Latitude_sample.data=High_Latitude_sample.data[High_Latitude_sample.data['ruwe']<1.75].reset_index(drop=True)

# Cuts

In [7]:
High_Latitude_sample.data=High_Latitude_sample.data.head(15)

In [8]:
High_Latitude_sample.data=High_Latitude_sample.data[High_Latitude_sample.data['logg_gspphot']>4.3].reset_index(drop=True)
High_Latitude_sample.data=High_Latitude_sample.data[High_Latitude_sample.data['logg_gspphot']<4.9].reset_index(drop=True)

In [9]:
High_Latitude_sample.data=High_Latitude_sample.data[High_Latitude_sample.data['r_med_geo']<1000.0].reset_index(drop=True)


In [10]:
#High_Latitude_sample.data=High_Latitude_sample.data[High_Latitude_sample.data['mh_gspphot']>-1.5].reset_index(drop=True)
stacked_isochrones=Isochrone_hl.stack_isochrones()
x_values=High_Latitude_sample.to_tensor(expandsize=stacked_isochrones.shape[-2])

# Posterior Inference Numpyro

In [11]:
import numpyro

In [12]:
import numpyro
from numpyro.infer import MCMC, NUTS, Predictive
import numpyro.distributions as dist
from jax import random
import jax.numpy as jnp

rng_key = random.PRNGKey(0)
assert numpyro.__version__.startswith("0.10.1")

In [13]:
x_values=jnp.array(x_values)

In [ ]:
stacked_isochrones=jnp.array(stacked_isochrones)

In [14]:
import math

In [15]:
x_columns=High_Latitude_sample.data.columns
isoc_columns=Isochrone_hl.isochrones_list[0].columns
def column_index(name):
    if name in x_columns:
        return np.where(np.array(x_columns)==name)[0][0]
    else:
        return np.where(np.array(isoc_columns)==name)[0][0] 

# Input data

In [16]:
m=math.atan(1.77)


g_obs=x_values[:,0,column_index('phot_g_mean_mag')]
g_obs_error=x_values[:,0,column_index('phot_g_mean_mag_error')]
C_obs=x_values[:,0,column_index('bp_rp')]
C_obs_error=x_values[:,0,column_index('bp_rp_error')]

feh_values=x_values[:,0,column_index('feh')]
feh_error=0.5

parallax_obs=x_values[:,0,column_index('parallax')]
parallax_obs_error=x_values[:,0,column_index('parallax_error')]

n_samples=len(x_values)
n_samples

3

In [17]:
x_columns=High_Latitude_sample.data.columns
isoc_columns=Isochrone_hl.isochrones_list[0].columns
def column_index(name):
    if name in x_columns:
        return np.where(np.array(x_columns)==name)[0][0]
    else:
        return np.where(np.array(isoc_columns)==name)[0][0] 

In [ ]:

def column_index_2(name):
    if name in x_columns:
        return np.where(np.array(x_columns)==name)[0][0]
    else:
        return np.where(np.array(isoc_columns)==name)[0][0]

In [ ]:
x_columns2=['G','bp_rp','teff_gspphot']
def column_index(name):
    if name in x_columns2:
        return np.where(np.array(x_columns2)==name)[0][0]
    else:
        return np.where(np.array(isoc_columns)==name)[0][0] + len(x_columns2)

In [18]:
for i in range(len(x_values)):
    if(i%500==0):
        print(i)
    G=jnp.array(x_values[i][0,column_index_2('G')])
    C=jnp.array(x_values[i][0,column_index_2('bp_rp')])
    teffmeas=jnp.array(x_values[i][0,column_index_2('teff_gspphot')])
    
    ones=jnp.ones_like((stacked_isochrones[:,:,column_index('G')]))
    single_source=jnp.concatenate([(G*ones)[:,:,jnp.newaxis],(C*ones)[:,:,jnp.newaxis],(teffmeas*ones)[:,:,jnp.newaxis]],axis=-1)

    isochrone=jnp.concatenate([single_source,stacked_isochrones],axis=-1)
    truth_1=(isochrone[:,:,column_index('G')]-(isochrone[:,:,column_index('bp_rp')])*isochrone[:,:,column_index('p_slopes')]<=isochrone[:,:,column_index('high_c')]) #box selection
    truth_2=(isochrone[:,:,column_index('low_c')]<=isochrone[:,:,column_index('G')]-(isochrone[:,:,column_index('bp_rp')])*isochrone[:,:,column_index('p_slopes')])
    truth=jnp.logical_and(truth_1,truth_2)


    a1=-1*isochrone[:,:,column_index('slopes')]
    b1=1.0
    c1=isochrone[:,:,column_index('slopes')]*isochrone[:,:,column_index('BPRP')]-isochrone[:,:,column_index('Gaia_G_EDR3')]


    a2=-1*isochrone[:,:,column_index('p_slopes')]
    b2=1.0
    c2=isochrone[:,:,column_index('p_slopes')]*isochrone[:,:,column_index('bp_rp')]-isochrone[:,:,column_index('G')]


    x=(b1*c2-b2*c1)/(a1*b2-a2*b1)
    y=(a2*c1-a1*c2)/(a1*b2-a2*b1)

    sgn=jnp.sign((y-isochrone[:,:,column_index('G')])/(x-isochrone[:,:,column_index('bp_rp')]))
    
    d=jnp.sqrt((x-isochrone[:,:,column_index('bp_rp')])**2+(y-isochrone[:,:,column_index('G')])**2)
    
    

   
    idx=(jnp.argmin(jnp.nan_to_num(jnp.abs(d/truth.reshape(d.shape)),nan=1e5),-1))
    #sgn=jnp.take(sgn,indices=idx)
    #sgn=jnp.array([sgn[i,idx[i]] for i in range(len(idx))])
    #d=jnp.take(d,indices=idx)*sgn
    #d=jnp.array([d[i,idx[i]] for i in range(len(idx))])*sgn

    #x_ints=jnp.take(x,indices=idx)#jnp.array([x[i,idx[i]] for i in range(len(idx))])     
    #y_ints=jnp.take(y,indices=idx)#jnp.array([y[i,idx[i]] for i in range(len(idx))])    
    #logg=jnp.take(isochrone[:,:,column_index('logg')],indices=idx)#jnp.array([isochrone[i,idx[i],column_index('logg')] for i in range(len(idx))])
    teff=jnp.take_along_axis(isochrone[:,:,column_index('logteff')],idx.reshape(len(isochrone[:,:,column_index('logteff')]),1),axis=-1)
    #teff=jnp.take(isochrone[:,:,column_index('logteff')],indices=idx)#jnp.array([isochrone[i,idx[i],column_index('logteff')] for i in range(len(idx))])
    #feh=jnp.take(isochrone[:,:,column_index('feh')],indices=idx)#jnp.array([isochrone[i,idx[i],column_index('feh')] for i in range(len(idx))])

    pt=jnp.argmin(jnp.abs(teff-jnp.log10(teffmeas)))
    #pt=jnp.argmin(jnp.abs(teff-jnp.log10(teffmeas)))

    #pt=np.argmin(np.abs(feh-(isochrone[0,0,column_index('feh_calib')].numpy()))) 
    if i == 0:
        stack=jnp.array(isochrone[pt,idx[pt],:])[jnp.newaxis,:]
    else: 
        stack=jnp.concatenate([stack,jnp.array(isochrone[pt,idx[pt],:])[jnp.newaxis,:]])

TypeError: unsupported operand type(s) for *: 'DeviceArray' and 'Tensor'

In [ ]:
plx_zp=-0.029
def model(g_obs,g_obs_error,C_obs,C_obs_error,parallax_obs,parallax_obs_error,feh_values,feh_error,n_samples):
    log_L = numpyro.sample("log_L", dist.Normal(np.log(370.0), 2.0))
    L = numpyro.deterministic("L", jnp.exp(log_L))
    with numpyro.plate("plate_i", n_samples):
        d_dist = dist.TransformedDistribution(
            dist.Chi2(6),
            dist.transforms.AffineTransform(
                0.0, 0.5 * L, domain=dist.constraints.positive
            ),
        )
        d=numpyro.sample('d',d_dist,rng_key=rng_key)
        ag=numpyro.sample('ag',dist.Uniform(low=0,high=5),rng_key=rng_key)       
        feh=numpyro.sample('feh',dist.TransformedDistribution(dist.Normal(0.0,1.0),dist.transforms.AffineTransform(-0.5,0.9,domain=dist.constraints.interval(-4.0,0.5)))) #could be anything

    #age_list=np.array([9.0])
    #stacked_isochrones=Isochrone_hl.stack_isochrones_subsample(feh.static(),age_list)
    G=jnp.array(stacked_isochrones[:,:,column_index('Gaia_G_EDR3')])
    G=jnp.nan_to_num(G,nan=jnp.inf)
    C=jnp.array(stacked_isochrones[:,:,column_index('BPRP')])
    C=jnp.nan_to_num(C,nan=jnp.inf)
    C=numpyro.deterministic("C",C+ag[:,jnp.newaxis]/1.77)
    g=numpyro.deterministic(
            "g", G +ag[:,jnp.newaxis]+5.0*jnp.log10(d[:,jnp.newaxis])-5.0
        )


    with numpyro.plate("data", n_samples):
        plx = numpyro.deterministic("plx", 1000.0 / d + plx_zp)
        numpyro.sample("plx_obs", dist.Normal(plx, parallax_obs_error), obs=parallax_obs)    
        numpyro.sample("feh_obs", dist.Normal(feh, feh_error), obs=feh_values)
        numpyro.sample('g_obs',dist.MultivariateNormal(g,g_obs_error[:,jnp.newaxis,jnp.newaxis]*jnp.eye(g.shape[-1])),obs=g_obs[:,jnp.newaxis]*jnp.ones(g.shape[-1])[jnp.newaxis,:])
        numpyro.sample('C_obs',dist.MultivariateNormal(C,C_obs_error[:,jnp.newaxis,jnp.newaxis]*jnp.eye(g.shape[-1])),obs=C_obs[:,jnp.newaxis]*jnp.ones(C.shape[-1])[jnp.newaxis,:])



In [ ]:
nuts_kernel = NUTS(model)

mcmc = MCMC(nuts_kernel, num_samples=2000, num_warmup=2000)
mcmc.run(rng_key, g_obs,g_obs_error,C_obs,C_obs_error,parallax_obs,parallax_obs_error,feh_values,feh_error,n_samples)

posterior_samples = mcmc.get_samples()

RuntimeError: Cannot find valid initial parameters. Please check your model again.